# Dot-heatmap of top eRs

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, sys
_stderr = sys.stderr
null = open(os.devnull, 'wb')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import dill
work_dir = '../BS1140'
scplus_obj = dill.load(open(os.path.join(work_dir, 'scenicplus/scplus_obj_4Pando.pkl'), 'rb')) # import scenic+ object

In [ ]:
# add eRegulon cluster celltype
import pandas as pd
eR_celltype = pd.read_csv(work_dir+'/cluster_overlap/eRegulon_celltype.csv', index_col=0)
eR_celltype.head()

,eRegulon_cluster
ATTCCTCCATAATCAC-1-BS1140,1-SP1+MAFF+
TGAAACTGTTAGAGGG-1-BS1140,3-ETS1+ZNF282+
TTGACATCAGTTTGTG-1-BS1140,1-SP1+MAFF+
TGCACTTGTTAGGTGC-1-BS1140,1-SP1+MAFF+
CCCAAACCAGGCAAGC-1-BS1140,8-CREM-BACH2-


In [ ]:
scplus_obj.metadata_cell['eR_celltype'] = eR_celltype # add to scenic+ object meta info

filter some high quality eRegulons to plot using the correlation between TF expression and target region enrichment scores (AUC values)

In [ ]:
# visualize the info on both TF genes expression and region accessibility
from scenicplus.cistromes import TF_cistrome_correlation, generate_pseudobulks

generate_pseudobulks(
        scplus_obj = scplus_obj,
        variable = 'eR_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_key = 'Gene_based')
generate_pseudobulks(
        scplus_obj = scplus_obj,
        variable = 'eR_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_key = 'Region_based')

TF_cistrome_correlation(
            scplus_obj,
            use_pseudobulk = True,
            variable = 'eR_celltype',
            auc_key = 'eRegulon_AUC_filtered',
            signature_key = 'Gene_based',
            out_key = 'filtered_gene_based')
TF_cistrome_correlation(
            scplus_obj,
            use_pseudobulk = True,
            variable = 'eR_celltype',
            auc_key = 'eRegulon_AUC_filtered',
            signature_key = 'Region_based',
            out_key = 'filtered_region_based')

/Users/jinhuixin/miniforge3/envs/scenicplus/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/jinhuixin/miniforge3/envs/scenicplus/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [ ]:
scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based'].head()

,TF,Cistrome,Rho,P-value,Adjusted_p-value
0,NFATC3,NFATC3_+_(18r),0.194411,5.669563e-10,9.785246e-10
1,BATF,BATF_-_(70r),-0.358076,1.284626e-31,3.837408e-31
2,TRPS1,TRPS1_-_(25r),-0.478360,2.567431e-58,1.616788e-57
3,REL,REL_-_(31r),0.022617,4.749794e-01,5.171504e-01
4,KLF2,KLF2_+_(265r),0.723980,3.988406e-163,1.858597e-161


### Calculate the RSS (regulon specificity score) of TFs

In [ ]:
from scenicplus.RSS import *
regulon_specificity_scores(
        scplus_obj,
        variable = 'eR_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_keys = ['Region_based'],
        selected_regulons = [x for x in scplus_obj.uns['selected_eRegulon']['Region_based'] if '-' not in x],
        out_key_suffix = '_filtered')

### Get top eR in each eRegulon cluster

In [ ]:
import scanpy as sc
adata = sc.read_h5ad('/Users/jinhuixin/Master/thesis/GRN/BS1140/eRegulon_cluster/eR_cluster_anndata.h5ad')

In [ ]:
eR_list = list()
for group in range(9): 
    eR_list += sc.get.rank_genes_groups_df(adata, group=str(group))['names'][:10].tolist()

In [ ]:
# gene-based and region-based top eR sets
selected_eRegulons_gene_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Gene_based'].keys()
        if x in eR_list]
selected_eRegulons_region_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Region_based'].keys()
        if x in eR_list]
#save the results in the scenicplus object
scplus_obj.uns['selected_eRegulon'] = {'Gene_based': selected_eRegulons_gene_sig, 'Region_based': selected_eRegulons_region_sig}
print(f'selected: {len(selected_eRegulons_gene_sig)} eRegulons')

selected: 34 eRegulons


modify some SCENIC+ functions

In [ ]:
import plotnine
from plotnine import ggplot, geom_point, aes, scale_fill_distiller, theme_bw, geom_tile, theme, element_text, element_blank
from plotnine.facets import facet_grid
def generate_dotplot_df_modified(
    scplus_obj: SCENICPLUS,
    size_matrix: pd.DataFrame,
    color_matrix: pd.DataFrame,
    scale_size_matrix: bool = True,
    scale_color_matrix: bool = True,
    group_variable: str = None,
    subset_eRegulons: list = None) -> pd.DataFrame:
    if all(np.isin(size_matrix.columns, scplus_obj.cell_names)):
        size_matrix = size_matrix.T
    if all(np.isin(size_matrix.index, scplus_obj.cell_names)):
        #calculate mean values
        if group_variable is None:
            raise ValueError('group_variable can not be None when size_matrix is a matrix with cell barcodes on one axis.')
        if group_variable not in scplus_obj.metadata_cell.columns:
            raise ValueError('group variable must be a column in scplus_obj.metadata_cell')
        size_matrix = size_matrix.groupby(scplus_obj.metadata_cell[group_variable]).mean()
    if all(np.isin(color_matrix.columns, scplus_obj.cell_names)):
        color_matrix = color_matrix.T
    if all(np.isin(color_matrix.index, scplus_obj.cell_names)):
        #calculate mean values
        if group_variable is None:
            raise ValueError('group_variable can not be None when color_matrix is a matrix with cell barcodes on one axis.')
        if group_variable not in scplus_obj.metadata_cell.columns:
            raise ValueError('group variable must be a column in scplus_obj.metadata_cell')
        color_matrix = color_matrix.groupby(scplus_obj.metadata_cell[group_variable]).mean()
    if not all(np.isin(size_matrix.index, color_matrix.index)):
        raise ValueError('size_matrix and color_matrix should have the same values as index.')
    size_matrix_features = (
        size_matrix.columns.to_list(),                      #full eRegulon name
        [f.split('_(')[0] for f in size_matrix.columns],    #eRegulon name without number of targets
        [f.split('_')[0] for f in size_matrix.columns])     #tf name
    color_matrix_features = (
        color_matrix.columns.to_list(),                      #full eRegulon name
        [f.split('_(')[0] for f in color_matrix.columns],    #eRegulon name without number of targets
        [f.split('_')[0] for f in color_matrix.columns])     #tf name

    #put features in same order
    #both matrices have eRegulons as features
    def _find_idx(l, e):
        return [i for i, v in enumerate(l) if v == e]
    if all(['_(' in x for x in size_matrix_features[0]]) and all(['_(' in x for x in color_matrix_features[0]]):
        if not all(np.isin(size_matrix_features[1], color_matrix_features[1])):
            raise ValueError('When two matrices are given with eRegulons as features then the names of these features (without number of targets) should match!')
        color_matrix = color_matrix.iloc[
            :, flatten_list([_find_idx(color_matrix_features[1], e) for e in size_matrix_features[1]])]
    #size matrix has eRegulons as features but color matrix not
    elif all(['_(' in x for x in size_matrix_features[0]]):
        color_matrix = color_matrix[size_matrix_features[2]]
    #color matrix has eRegulons as features but size matrix not
    elif all(['_(' in x for x in color_matrix_features[0]]):
        size_matrix = size_matrix[color_matrix_features[1]]
    #none of the matrices have eRegulons as features
    else:
        size_matrix = size_matrix[color_matrix_features[1]]
    
    if subset_eRegulons is not None:
        #change to TF names
        subset_eRegulons = [x.split('_(')[0] for x in subset_eRegulons]
        size_matrix = size_matrix[[x for x in size_matrix if x.split('_(')[0] in subset_eRegulons]]
        subset_eRegulons_TF = [x.split('_')[0] for x in subset_eRegulons]
        color_matrix = color_matrix[[x for x in color_matrix if x in subset_eRegulons_TF]]
    
    if scale_size_matrix:
        size_matrix = (size_matrix - size_matrix.min()) / (size_matrix.max() - size_matrix.min())
    if scale_color_matrix:
        color_matrix = (color_matrix - color_matrix.min()) / (color_matrix.max() - color_matrix.min())
    
    size_matrix_df = size_matrix.stack().reset_index()
    color_matrix_df = color_matrix.stack().reset_index()
    size_matrix_df.columns = ['index', 'size_name', 'size_val']
    color_matrix_df.columns = ['index', 'color_name', 'color_val']
    if all(['_(' in x for x in size_matrix_features[0]]) and all(['_(' in x for x in color_matrix_features[0]]):
        size_matrix_df['eRegulon_name'] = [x.split('_(')[0] for x in size_matrix_df['size_name']]
        color_matrix_df['eRegulon_name'] = [x.split('_(')[0] for x in color_matrix_df['color_name']]
        merged_df = size_matrix_df.merge(color_matrix_df, on = ['index', 'eRegulon_name'])
        merged_df['TF'] = [x.split('_')[0] for x in merged_df['eRegulon_name']]
    else:
        size_matrix_df['TF'] = [x.split('_')[0] for x in size_matrix_df['size_name']]
        color_matrix_df['TF'] = [x.split('_')[0] for x in color_matrix_df['color_name']]
        merged_df = size_matrix_df.merge(color_matrix_df, on = ['index', 'TF'])
        if all(['_(' in x for x in size_matrix_features[0]]):
            merged_df['eRegulon_name'] = merged_df['size_name']
        elif all(['_(' in x for x in color_matrix_features[0]]):
            merged_df['eRegulon_name'] = merged_df['color_name']
        else:
            merged_df['eRegulon_name'] = merged_df['TF']

    #for esthetics
    merged_df = merged_df[['index', 'TF', 'eRegulon_name', 'size_name', 'color_name', 'size_val', 'color_val']]
    return merged_df

In [ ]:
def heatmap_dotplot_modified(
    scplus_obj: SCENICPLUS,
    size_matrix: pd.DataFrame,
    color_matrix: pd.DataFrame,
    scale_size_matrix: bool = True,
    scale_color_matrix: bool = True,
    group_variable: str = None,
    subset_eRegulons: list = None,
    sort_by: str = 'color_val',
    index_order: list = None,
    save: str = None,
    figsize: tuple = (5, 8),
    split_repressor_activator: bool = True,
    orientation: str = 'vertical'):
    plotting_df = generate_dotplot_df_modified(
        scplus_obj = scplus_obj,
        size_matrix = size_matrix,
        color_matrix = color_matrix,
        scale_size_matrix = scale_size_matrix,
        scale_color_matrix = scale_color_matrix,
        group_variable = group_variable,
        subset_eRegulons = subset_eRegulons)
    if index_order is not None:
        if len(set(index_order) & set(plotting_df['index'])) != len(set(plotting_df['index'])):
            Warning('not all indices are provided in index_order, order will not be changed!')
        else:
            plotting_df['index'] = pd.Categorical(plotting_df['index'], categories = index_order)
    #sort values
    tmp = plotting_df[['index', 'eRegulon_name', sort_by]
        ].pivot_table(index = 'index', columns = 'eRegulon_name'
        ).fillna(0)['color_val']
    if index_order is not None:
        tmp = tmp.loc[index_order]
    idx_max = tmp.idxmax(axis = 0)
    order = pd.concat([idx_max[idx_max == x] for x in tmp.index.tolist() if len(plotting_df[plotting_df == x]) > 0]).index.tolist()
    plotting_df['eRegulon_name'] = pd.Categorical(plotting_df['eRegulon_name'], categories = order)
    plotnine.options.figure_size = figsize
    if split_repressor_activator:
        plotting_df['repressor_activator'] = ['activator' if '+' in n.split('_')[1] and 'extended' not in n or '+' in n.split('_')[2] and 'extended' in n  else 'repressor' for n in plotting_df['eRegulon_name']]
        if orientation == 'vertical':
            plot = (
                ggplot(plotting_df, aes('index', 'eRegulon_name'))
                + facet_grid(
                    'repressor_activator ~ .', 
                    scales = "free", 
                    space = {'x': [1], 'y': [sum(plotting_df['repressor_activator'] == 'activator'), sum(plotting_df['repressor_activator'] == 'repressor')]})
                + geom_tile(mapping = aes(fill = 'color_val'))
                + scale_fill_distiller(type = 'div', palette = 'RdYlBu')
                + geom_point(
                        mapping = aes(size = 'size_val'),
                        colour = "black")
                + theme(axis_text_x=element_text(rotation=90, hjust=1))
                + theme(axis_title_x = element_blank(), axis_title_y = element_blank()))
        elif orientation == 'horizontal':
            plot = (
                ggplot(plotting_df, aes('eRegulon_name', 'index'))
                + facet_grid(
                    '. ~ repressor_activator', 
                    scales = "free", 
                    space = {'y': [1], 'x': [sum(plotting_df['repressor_activator'] == 'activator'), sum(plotting_df['repressor_activator'] == 'repressor')]})
                + geom_tile(mapping = aes(fill = 'color_val'))
                + scale_fill_distiller(type = 'div', palette = 'RdYlBu')
                + geom_point(
                        mapping = aes(size = 'size_val'),
                        colour = "black")
                + theme(axis_text_x=element_text(rotation=90, hjust=1))
                + theme(axis_title_x = element_blank(), axis_title_y = element_blank()))
    else:
        if orientation == 'vertical':
            plot = (
                ggplot(plotting_df, aes('index', 'eRegulon_name'))
                + geom_tile(mapping = aes(fill = 'color_val'))
                + scale_fill_distiller(type = 'div', palette = 'RdYlBu')
                + geom_point(
                        mapping = aes(size = 'size_val'),
                        colour = "black")
                + theme(axis_title_x = element_blank(), axis_title_y = element_blank()))
        elif orientation == 'horizontal':
            plot = (
                ggplot(plotting_df, aes('eRegulon_name', 'index'))
                + geom_tile(mapping = aes(fill = 'color_val'))
                + scale_fill_distiller(type = 'div', palette = 'RdYlBu')
                + geom_point(
                        mapping = aes(size = 'size_val'),
                        colour = "black")
                + theme(axis_title_x = element_blank(), axis_title_y = element_blank()))
    if save is not None:
        plot.save(save)
    else:
        return plot

In [ ]:
# gene-based dot-heatmap of the set of top eRs in all clusters
heatmap_dotplot_modified(
        scplus_obj = scplus_obj,
        size_matrix = scplus_obj.uns['eRegulon_AUC_filtered']['Gene_based'], #specify what to plot as dot sizes, target gene expression in this case
        color_matrix = scplus_obj.to_df('EXP'), #specify  what to plot as colors, TF expression in this case
        scale_size_matrix = True,
        scale_color_matrix = True,
        group_variable = 'eR_celltype',
        subset_eRegulons = selected_eRegulons_gene_sig,
        index_order = ['0-ZSCAN22+EOMES+', '1-SP1+MAFF+', '2-FOXP1+ARID5B+', '3-ETS1+ZNF282+', '4-ZNF442+ZNF526+', '5-TBX21+RARG+', '6-CREM-ELF1+', '7-ZNF385D+RORC+', '8-CREM-BACH2-', '9-EGR1+FOSB+'],
        figsize = (5, 20),
        orientation = 'vertical',
        save = work_dir+'/eRegulon_cluster/heatdotmap_gene_based_top_eRegulon_each_cluster.pdf'
        )

/Users/jinhuixin/miniforge3/envs/scenicplus/lib/python3.9/site-packages/plotnine/ggplot.py:718: PlotnineWarning: Saving 5 x 20 in image.
/Users/jinhuixin/miniforge3/envs/scenicplus/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Filename: ../BS1140/eRegulon_cluster/heatdotmap_gene_based_top_eRegulon_each_cluster.pdf


In [ ]:
# region-based dot-heatmap of the set of top eRs in all clusters
heatmap_dotplot_modified(
        scplus_obj = scplus_obj,
        size_matrix = scplus_obj.uns['eRegulon_AUC_filtered']['Region_based'], #specify what to plot as dot sizes, target gene expression in this case
        color_matrix = scplus_obj.to_df('EXP'), #specify  what to plot as colors, TF expression in this case
        scale_size_matrix = True,
        scale_color_matrix = True,
        group_variable = 'eR_celltype',
        subset_eRegulons = selected_eRegulons_region_sig,
        index_order = ['0-ZSCAN22+EOMES+', '1-SP1+MAFF+', '2-FOXP1+ARID5B+', '3-ETS1+ZNF282+', '4-ZNF442+ZNF526+', '5-TBX21+RARG+', '6-CREM-ELF1+', '7-ZNF385D+RORC+', '8-CREM-BACH2-', '9-EGR1+FOSB+'],
        figsize = (5, 20),
        orientation = 'vertical',
        save = work_dir+'/eRegulon_cluster/heatdotmap_region_based_top_eRegulon_each_cluster.pdf'
        )

/Users/jinhuixin/miniforge3/envs/scenicplus/lib/python3.9/site-packages/plotnine/ggplot.py:718: PlotnineWarning: Saving 5 x 20 in image.
/Users/jinhuixin/miniforge3/envs/scenicplus/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Filename: ../BS1140/eRegulon_cluster/heatdotmap_region_based_top_eRegulon_each_cluster.pdf
